# Assessment: Analyzing Soccer Data

TODO: add intro

## 0. Set-up

### a. Load linting tool, create spark session, etc.

In [1]:
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle
from pyspark.sql import SparkSession
import pyspark.sql.functions as func

# Create spark app
spark = (
    SparkSession
    .builder
    .appName("rb assessment app")
    .getOrCreate()
)


### a. Data Engineering for Task 1

In [3]:
#%%pycodestyle
import pyspark.sql.functions as func

bl_results_spark_df = (
    # read matches folder
    spark
    .read
    .json(
        "data/matches/*/",
        multiLine=True
    )
    # filter 1. Bundesliga 2015/16 matches
    .where(
        func.col("competition.competition_name").eqNullSafe("1. Bundesliga")
        & func.col("season.season_name").eqNullSafe("2015/2016")
    )
    # order by date to make sure that the ELO function is used approprietly
    .orderBy(func.to_date("match_date"))
    # perform the mapping to get one row for each team and match
    # the associated result is:
    # - win = 1
    # - draw = 0.5
    # - loss = 0
    .select(
        func.col(
            "home_team.home_team_name"
        ).alias("home_team_name"),
        func.col(
            "away_team.away_team_name"
        ).alias("away_team_name"),
        func.coalesce(
            func.when(func.expr("home_score > away_score"), 1),
            func.when(func.expr("home_score < away_score"), 0),
            func.lit(0.5)
        ).alias("home_team_result"),
        func.coalesce(
            func.when(func.expr("home_score > away_score"), 0),
            func.when(func.expr("home_score < away_score"), 1),
            func.lit(0.5)
        ).alias("away_team_result")
    )
)

# print schema for verification
bl_results_spark_df.printSchema()

# cache result to pandas df
bl_result_df = bl_results_spark_df.toPandas()
# display resulting df
print(bl_result_df)

root
 |-- home_team_name: string (nullable = true)
 |-- away_team_name: string (nullable = true)
 |-- home_team_result: double (nullable = false)
 |-- away_team_result: double (nullable = false)

    home_team_name            away_team_name  home_team_result  \
0    Bayern Munich              Hamburger SV               1.0   
1         Augsburg             Hertha Berlin               0.0   
2    Werder Bremen                Schalke 04               0.0   
3     FSV Mainz 05                Ingolstadt               0.0   
4     Darmstadt 98               Hannover 96               0.5   
..             ...                       ...               ...   
301   Darmstadt 98  Borussia Mönchengladbach               0.0   
302  Bayern Munich               Hannover 96               1.0   
303   FSV Mainz 05             Hertha Berlin               0.5   
304  Werder Bremen       Eintracht Frankfurt               1.0   
305      Wolfsburg             VfB Stuttgart               1.0   

     away_t

## 1. Task 1 - Elo Rating

### a. Develop a function to implement the ELO Rating System with arbitrary K and s.

In [4]:
#%%pycodestyle
import math
import operator


def predict(rating_a, rating_b, s=15):
    """Calculate the expected outcome for team A given ratings."""
    return 1 / (1 + math.pow(10, -(rating_a - rating_b) / s))


def update_elo(rating_a, rating_b, outcome, s=15, K=15):
    """Update the ELO rating of a team after a match."""
    pred = predict(rating_a, rating_b, s)
    return rating_a + K * (outcome - pred)


def process_match(ratings, home_team, away_team, home_result, away_result,
                  s, K):
    """Update ratings after processing a match."""
    # Update home team rating
    ratings[home_team] = update_elo(
        ratings[home_team], ratings[away_team], home_result, s, K
    )
    # Update away team rating
    ratings[away_team] = update_elo(
        ratings[away_team], ratings[home_team], away_result, s, K
    )
    return ratings


def initialize_ratings(teams, initial_rating=100):
    """Initialize the ratings for all teams."""
    return {team_name: initial_rating for team_name in teams}


def elo(results, s=15, K=15, R_0=100):
    """Calculate the final ELO ratings for all teams."""
    # Initialize ratings
    ratings = initialize_ratings(results["home_team_name"].unique(), R_0)
    for row in results.itertuples(index=False):
        # Update ratings based on the match result
        process_match(
            ratings,
            row.home_team_name,
            row.away_team_name,
            row.home_team_result,
            row.away_team_result,
            s, K
        )
    # Sort the ratings in descending order
    return dict(
        sorted(ratings.items(), key=operator.itemgetter(1), reverse=True)
    )


### b. Apply the rating system to 1. Bundesliga 2015/2016 Season with starting values R0 = 100, s = 15 and K = 15.

In [5]:
elo(bl_result_df)

{'Borussia Mönchengladbach': 128.73684750507124,
 'Bayern Munich': 126.03882090784246,
 'Werder Bremen': 122.53431008967196,
 'Bayer Leverkusen': 119.86744451903735,
 'Schalke 04': 116.7404982618847,
 'Eintracht Frankfurt': 115.13792895836448,
 'Borussia Dortmund': 113.14276236567889,
 'FC Köln': 112.7931080154221,
 'Hannover 96': 112.683637686516,
 'Darmstadt 98': 104.6708369599969,
 'Hoffenheim': 104.14444368636025,
 'Ingolstadt': 104.05781546247819,
 'Hamburger SV': 103.82508894341434,
 'Wolfsburg': 103.54107736006338,
 'Augsburg': 100.16514926109396,
 'FSV Mainz 05': 98.21672472241177,
 'Hertha Berlin': 96.401623690483,
 'VfB Stuttgart': 88.0062841309003}

### c. Develop an approach that finds the optimal values for s and K based on that season and display the final ranking table at the end of the season.

We could take the assumption that the optimal values for parameters s and K are the one that would minimize the discrepancy between the predicted outcome and actual match result for each step of that particular season.

Let's define our loss function as the average brier score of our ELO Rating System.

In [6]:
#%%pycodestyle

def brier_score(results, s=15, K=15, R_0=100):
    """
    Calculate the Brier score based on the predicted outcomes and actual
    results.
    """
    ratings = initialize_ratings(results["home_team_name"].unique(), R_0)
    total_brier_score = 0
    for row in results.itertuples(index=False):
        # Get teams predictions
        home_rating = ratings[row.home_team_name]
        away_rating = ratings[row.away_team_name]

        home_pred = predict(home_rating, away_rating, s)
        away_pred = 1 - home_pred

        # Calculate Brier score for both teams
        total_brier_score += (home_pred - row.home_team_result) ** 2
        total_brier_score += (away_pred - row.away_team_result) ** 2

        # Update ratings after the match
        process_match(
            ratings,
            row.home_team_name,
            row.away_team_name,
            row.home_team_result,
            row.away_team_result,
            s, K
        )

    # Return the average Brier score
    return total_brier_score / (2 * len(results))

brier_score(bl_result_df)


0.23717873793811076

We are going to try finding good values for s and K params for brier_score in terms of brier score using bayesian optimization.
It is a sampling method based on exploration and optimization.

In [17]:
#%%pycodestyle
from functools import partial
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'s': (15, 300), 'K': (15, 30)}

def neg_brier_score(s, K):
    return - brier_score(bl_result_df, s=s, K=K)

optimizer = BayesianOptimization(
    f=neg_brier_score,
    pbounds=pbounds,
    random_state=1,
    verbose=1
)

optimizer.maximize(
    init_points=32,
    n_iter=128,
)

|   iter    |  target   |     K     |     s     |
-------------------------------------------------
| 7         | -0.178    | 18.07     | 265.3     |
| 8         | -0.1779   | 15.41     | 206.1     |
| 44        | -0.1779   | 22.83     | 300.0     |
| 59        | -0.1779   | 15.0      | 200.4     |
| 94        | -0.1779   | 18.18     | 241.4     |
| 130       | -0.1779   | 20.45     | 271.1     |


In [18]:
res = optimizer.max
print(res)

{'target': np.float64(-0.17791904423717828), 'params': {'K': np.float64(20.44873714712105), 's': np.float64(271.1405558615955)}}


In [19]:
elo(bl_result_df, s=res["params"]["s"], K=res["params"]["K"])

{'Bayern Munich': np.float64(239.10714701168303),
 'Borussia Dortmund': np.float64(202.74555857602832),
 'Bayer Leverkusen': np.float64(156.15384473399382),
 'Borussia Mönchengladbach': np.float64(135.36332335175678),
 'Schalke 04': np.float64(117.98494348740996),
 'FSV Mainz 05': np.float64(106.89958701018837),
 'FC Köln': np.float64(95.41208312462189),
 'Hertha Berlin': np.float64(95.35379411470757),
 'Wolfsburg': np.float64(83.21128855250517),
 'Augsburg': np.float64(81.1916679150901),
 'Hoffenheim': np.float64(80.0920946039107),
 'Werder Bremen': np.float64(77.97082248179228),
 'Hamburger SV': np.float64(74.23642223799513),
 'Darmstadt 98': np.float64(71.19245828494051),
 'Ingolstadt': np.float64(71.0165776348987),
 'Eintracht Frankfurt': np.float64(70.6169902683531),
 'VfB Stuttgart': np.float64(37.66955722879909),
 'Hannover 96': np.float64(19.194387968792206)}

As we can see, the final ranking at the end of the season is strongly correlated with the actual league table.
But, we could try optimizing the parameters a bit further using gradient descent from those starting parameters.
Gradient descent is a standard practice in machine learning to find parameters that allow a function to reach a minimum in terms of loss.

In [35]:
#%%pycodestyle
import torch
from torch.optim import Adam


# Define the PyTorch version of predict
def predict_torch(rating_a, rating_b, s):
    return 1 / (1 + torch.pow(10, -(rating_a - rating_b) / s))


# Define the PyTorch version of update_elo
def update_elo_torch(rating_a, rating_b, outcome, s, K):
    pred = predict_torch(rating_a, rating_b, s)
    return rating_a + K * (outcome - pred)


# Convert the brier_score function to work with PyTorch tensors
def brier_score_torch(results_df, s, K):
    # Initialize ratings with PyTorch tensors
    unique_teams = results_df["home_team_name"].unique()
    ratings = {
        team_name: torch.tensor(100.0, requires_grad=False)
        for team_name in unique_teams
    }

    total_brier_score = torch.tensor(0.0, dtype=torch.float32)

    for row in results_df.itertuples(index=False):
        home_rating = ratings[row.home_team_name]
        away_rating = ratings[row.away_team_name]

        # Use the torch-based predict function
        home_pred = predict_torch(home_rating, away_rating, s)
        away_pred = 1 - home_pred

        home_team_result = torch.tensor(
            [row.home_team_result], dtype=torch.float32
        )
        away_team_result = torch.tensor(
            [row.away_team_result], dtype=torch.float32
        )

        # Accumulate Brier score and ensure correct shape
        total_brier_score = total_brier_score \
            + torch.pow(home_pred - home_team_result, 2)
        total_brier_score = total_brier_score \
            + torch.pow(away_pred - away_team_result, 2)

        # Update ratings with torch-based update function
        ratings[row.home_team_name] = update_elo_torch(
            home_rating, away_rating, row.home_team_result, s, K
        )
        ratings[row.away_team_name] = update_elo_torch(
            away_rating, home_rating, row.away_team_result, s, K
        )

    # Return the average Brier score as a tensor
    return total_brier_score / (2 * len(results_df))


# Initial values for s and K as torch tensors (requires gradients)
s = torch.tensor([res["params"]["s"]], requires_grad=True)
K = torch.tensor([res["params"]["K"]], requires_grad=True)

# Define the optimizer
optimizer = Adam([s, K], lr=0.001)

# Number of gradient descent steps
n_iterations = 500

# Perform gradient descent
for i in range(n_iterations):
    optimizer.zero_grad()  # Zero the gradients from the previous iteration

    # Calculate the Brier score
    loss = brier_score_torch(bl_result_df, s, K)

    # Backpropagation to compute gradients
    loss.backward()

    # Perform a step of gradient descent
    optimizer.step()

    # Optionally, print progress
    if i % 100 == 0:
        print(f"Iteration {i}: Brier score = {loss.item()}, s = {s.item()}, K = {K.item()}")

# Final optimized values of s and K
print(f"Optimized s: {s.item()}, Optimized K: {K.item()}")


Iteration 0: Brier score = 0.17795345066574975, s = 271.14154310709904, K = 20.447738120511808
Iteration 100: Brier score = 0.17795254370403202, s = 271.2349039633821, K = 20.35309214544007
Iteration 200: Brier score = 0.17795199812485302, s = 271.3126315462579, K = 20.273932400837435
Iteration 300: Brier score = 0.17795170632470167, s = 271.3735168370143, K = 20.21162764201154
Iteration 400: Brier score = 0.17795156975407675, s = 271.4180023744578, K = 20.16588990044537
Optimized s: 271.4477628525792, Optimized K: 20.135156178137716


In [23]:
elo(bl_result_df, s=s.item(), K=K.item())

{'Bayern Munich': 238.11614695578442,
 'Borussia Dortmund': 202.10731690794827,
 'Bayer Leverkusen': 155.58162112932683,
 'Borussia Mönchengladbach': 134.8976205950394,
 'Schalke 04': 117.80211535227875,
 'FSV Mainz 05': 106.9566223178877,
 'Hertha Berlin': 95.6603558234831,
 'FC Köln': 95.37158298186573,
 'Wolfsburg': 83.44786249701849,
 'Augsburg': 81.22629511106862,
 'Hoffenheim': 80.08479250444809,
 'Werder Bremen': 77.92219184991505,
 'Hamburger SV': 74.46172196837092,
 'Darmstadt 98': 71.39588595656109,
 'Ingolstadt': 71.31005191484432,
 'Eintracht Frankfurt': 70.64756396595439,
 'VfB Stuttgart': 38.3064811665141,
 'Hannover 96': 19.632316449702756}

Gradient descent does not change the final rankings, but it found a slightly lower K value and higher s value indincating slower ELO updates and more conservative outcome predictions.

## 2. Task 2 - Research Problem Corners

### a. Use the whole data set of Free Statsbomb Data to help you find general offensive trends.

In [49]:
# end_location -> cat
# body_part -> inswinging, outswinging
# 

df.select("pass.*").printSchema()

root
 |-- aerial_won: boolean (nullable = true)
 |-- angle: double (nullable = true)
 |-- assisted_shot_id: string (nullable = true)
 |-- backheel: boolean (nullable = true)
 |-- body_part: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |-- cross: boolean (nullable = true)
 |-- cut_back: boolean (nullable = true)
 |-- deflected: boolean (nullable = true)
 |-- end_location: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- goal_assist: boolean (nullable = true)
 |-- height: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |-- inswinging: boolean (nullable = true)
 |-- length: double (nullable = true)
 |-- miscommunication: boolean (nullable = true)
 |-- no_touch: boolean (nullable = true)
 |-- outcome: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |-- outswinging: boolean (nullable = true)
 |-